In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import FloatVGG16
from SampleNN import *
from tqdm import tqdm_notebook
%matplotlib inline
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision

In [2]:
import torchvision.transforms as transforms
traindir = '/home/data/yanzy/train'
valdir = '/home/data/yanzy/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_dataset =  torchvision.datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                         shuffle=False, num_workers = 32, pin_memory=False)

train_dataset =  torchvision.datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,
                                         shuffle=True, num_workers = 16, pin_memory=False)

In [3]:
import SampleVGG16
sample_net = SampleVGG16.SampleVGG16().to(device)
float_net = torchvision.models.vgg16(pretrained=True).to(device)
lol = sample_net.state_dict()

a = lol.keys()
print len(a)
gt  = float_net.state_dict()
b = gt.keys()
for i in  range(32):
    lol[a[i]] = gt[b[i]]
sample_net.load_state_dict(lol)

32


In [4]:
import SampleVGG16
sample_net = SampleVGG16.SampleVGG16().to(device)

In [4]:
def val(Sample = 1):
    TMP = protectStateDict(sample_net)
    sampleStateDict(sample_net)
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader):
            if count % Sample == 0:
            
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                images = oSample()(images)
                outputs = sample_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)


                correct += (predicted == labels).sum().item()
                #print correct, total
            count += 1

    print('Accuracy of the network on the %d test images: %.3f %%' % (total,
        100.0 * correct / total))
    
    sample_net.load_state_dict(TMP)
    return 100.0 * correct / total

In [5]:
criterion_s = nn.CrossEntropyLoss()
#optimizer_s = optim.SGD(sample_net.parameters(), lr=1e-6, momentum=0.9)
optimizer_s = optim.Adam(sample_net.parameters(), lr=1e-5, weight_decay=1e-6)
def train(epoch, Sample = 10000):
    sample_net.train()
    running_loss = 0.0
    count = 0
    for i, data in tqdm_notebook(enumerate(train_loader, 0)):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer_s.zero_grad()

        # forward + backward + optimize
        TMP = protectStateDict(sample_net)
        #sampleStateDict(sample_net)
        outputs = sample_net(inputs)
        loss = criterion_s(outputs, labels)
        loss.backward()
        sample_net.load_state_dict(TMP)
        del TMP
        torch.cuda.empty_cache()
        optimizer_s.step()
        count += labels.size(0)
        if count >= Sample:
            break

        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 1000))
    return running_loss

In [6]:
val(Sample = 8)


Accuracy of the network on the 6272 test images: 28.699 %
1800


28.698979591836736

In [9]:
for epoch in range(4):

    torch.cuda.empty_cache()
    train(epoch, Sample = 10000000)
    torch.cuda.empty_cache()
    val(Sample = 1)

/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18481152 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 37093376 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 39976960 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/


[1, 80073] loss: 122.346



Accuracy of the network on the 50000 test images: 54.130 %
27065


/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18481152 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/lo


[2, 80073] loss: 121.856


...


[3, 80073] loss: 121.579



Accuracy of the network on the 50000 test images: 54.646 %
27323


/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:764: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/yanzy/.virtualenvs/fp/local/lib/python2.7/site-packages/PIL/TiffImagePlugin.


[4, 80073] loss: 121.269



Accuracy of the network on the 50000 test images: 54.938 %
27469
